In [1]:
#|hide
#|eval: false
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [4]:
#|export
# Python native modules
import os
from torch.multiprocessing import Queue
# Third party libs
from fastcore.all import *
import torchdata.datapipes as dp
# Local modules
from fastrl.loggers.core import *
from fastai.torch_basics import*

In [5]:
#|hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [6]:
#|default_exp loggers.tensorboard

# Tensorboard 
> Iterable pipes for exporting to tensorboard

In [7]:
#|export
def run_tensorboard(
        port:int=6006, # The port to run tensorboard on/connect on
        start_tag:str=None, # Starting regex e.g.: experience_replay/1
        samples_per_plugin:str=None, # Sampling freq such as  images=0 (keep all)
        extra_args:str=None, # Any additional arguments in the `--arg value` format
        rm_glob:bool=None # Remove old logs via a parttern e.g.: '*' will remove all files: runs/* 
    ):
    if rm_glob is not None:
        for p in Path('runs').glob(rm_glob): p.delete()
    import socket
    from tensorboard import notebook
    a_socket=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    cmd=None
    if not a_socket.connect_ex(('127.0.0.1',6006)):
        notebook.display(port=port,height=1000)
    else:
        cmd=f'--logdir runs --port {port} --host=0.0.0.0'
        if samples_per_plugin is not None: cmd+=f' --samples_per_plugin {samples_per_plugin}'
        if start_tag is not None:          cmd+=f' --tag {start_tag}'
        if extra_args is not None:         cmd+=f' {extra_args}'
        notebook.start(cmd)
    return cmd

In [8]:
#|hide
SHOW_TENSOR_BOARD=False
if not os.environ.get("IN_TEST", None) and SHOW_TENSOR_BOARD:
    run_tensorboard()

In [9]:
#|hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()